In [3]:
import re

texto = "¡Hola! ¿Cómo estás? Este es mi número: 555-777-888"
texto = texto.lower() # Minúscula 
texto = re.sub(r"[\W\d]", " ", texto) # Dejar sólo letras
texto = texto.strip()
texto = texto.split() # Tokenizar
texto


['hola', 'cómo', 'estás', 'este', 'es', 'mi', 'número']

In [1]:
# !pip install nltk
# import nltk
# nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_sp = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SebastianVargasGanti\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [4]:
texto = [palabra for palabra in texto if palabra not in stopwords_sp]
texto

['hola', 'cómo', 'número']

In [5]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer = SnowballStemmer("english")
spanishStemmer = SnowballStemmer("spanish")

In [6]:
englishStemmer.stem("car")
englishStemmer.stem("cars")

spanishStemmer.stem("cantar")
spanishStemmer.stem("cantamos")

'cant'

In [8]:
texto_stem = []
for palabra in texto:
    raiz = spanishStemmer.stem(palabra)
    texto_stem.append(raiz)
    
texto_stem


['hol', 'com', 'numer']

In [10]:
import nltk 
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet') # Sólo se corre la primera vez

lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("dancing", pos="v")

'dance'

In [12]:
import es_core_news_sm

nlp = es_core_news_sm.load()
doc = nlp("¡Hola! ¿Cómo estás? Este es mi número 314-1592. ¡Vamos a bailar porque es jueves!")
[(palabra.text, palabra.pos_) for palabra in doc]

[('¡', 'PUNCT'),
 ('Hola', 'PROPN'),
 ('!', 'PUNCT'),
 ('¿', 'PUNCT'),
 ('Cómo', 'PRON'),
 ('estás', 'VERB'),
 ('?', 'PUNCT'),
 ('Este', 'PRON'),
 ('es', 'AUX'),
 ('mi', 'DET'),
 ('número', 'NOUN'),
 ('314-1592', 'NUM'),
 ('.', 'PUNCT'),
 ('¡', 'PUNCT'),
 ('Vamos', 'AUX'),
 ('a', 'ADP'),
 ('bailar', 'VERB'),
 ('porque', 'SCONJ'),
 ('es', 'AUX'),
 ('jueves', 'NOUN'),
 ('!', 'PUNCT')]

In [13]:
from spacy import displacy 

displacy.render(doc, style="dep", jupyter=True, options = {'distance':150})

In [24]:
import pandas as pd 
import numpy as np

corpus = {'D1': 'perro muerde hombre',
          'D2': 'hombre muerde perro',
          'D3': 'perro come carne',
          'D4': 'hombre come comida'}

corpus = pd.DataFrame.from_dict(corpus, orient='index', columns=['texto'])
corpus

,texto
D1,perro muerde hombre
D2,hombre muerde perro
D3,perro come carne
D4,hombre come comida


In [31]:
vocabulario = corpus['texto'].str.cat(sep=" ")
vocabulario = set(vocabulario.split())
vocabulario = [(palabra,i) for i,palabra in enumerate(vocabulario)]
print(vocabulario)

[('carne', 0), ('perro', 1), ('muerde', 2), ('comida', 3), ('come', 4), ('hombre', 5)]


In [32]:
vocab_onehot = np.array([[0]*len(vocabulario)]*len(vocabulario))
# vocab_onehot
for palabra,i in vocabulario:
    vocab_onehot[i,i] = 1
    # print(f"Palabra: {palabra}")
    # print(f"One-hot encoding: {vocab_onehot[i,:]}")
    # print()
    
vocab_onehot = pd.DataFrame(vocab_onehot)
vocab_onehot.columns = [palabra[0] for palabra in vocabulario]
vocab_onehot.index = [palabra[0] for palabra in vocabulario]
vocab_onehot

,carne,perro,muerde,comida,come,hombre
carne,1,0,0,0,0,0
perro,0,1,0,0,0,0
muerde,0,0,1,0,0,0
comida,0,0,0,1,0,0
come,0,0,0,0,1,0
hombre,0,0,0,0,0,1


In [33]:
def one_hot_encoder(frase):
    frase_onehot = []
    for palabra in frase.split():
        frase_onehot.append(vocab_onehot[palabra].tolist())
    print(frase_onehot)
    
    
    
one_hot_encoder("perro muerde hombre")

[[0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1]]


In [34]:
def bow(frase):
    frase_bow = [0]*len(vocabulario)
    for palabra,i in vocabulario:
        if palabra in frase.split():
            frase_bow[i] = 1
    print(frase_bow)
    
    
bow("perro muerde hombre")

[0, 1, 1, 0, 0, 1]


In [38]:
np.sum(np.array([[0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1]]),0)

array([0, 1, 1, 0, 0, 1])

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer() # CountVectorizer(binary=True) si no nos importa la frecuencia
bow_rep = count_vect.fit_transform(corpus.texto.values)

print("Vocabulario: ", count_vect.vocabulary_)

# print(corpus.iloc[0])
# print(bow_rep[0].toarray())

corpus['bow'] = [row for row in bow_rep.toarray()]
corpus

Vocabulario:  {'perro': 5, 'muerde': 4, 'hombre': 3, 'come': 1, 'carne': 0, 'comida': 2}


,texto,bow
D1,perro muerde hombre,"[0, 0, 0, 1, 1, 1]"
D2,hombre muerde perro,"[0, 0, 0, 1, 1, 1]"
D3,perro come carne,"[1, 1, 0, 0, 0, 1]"
D4,hombre come comida,"[0, 1, 1, 1, 0, 0]"
